In [5]:
import numpy as np

In [10]:
#criação do sistema de equações de balanço de massa dos estágios na forma matricial
# fazer sem usar o numpy
def junta_matriz(matriz, estagio):    
    if estagio > 2:
        return np.concatenate((junta_matriz (matriz, estagio - 1), matriz[estagio - 1]), axis = 0)        
    else:
        return np.concatenate((matriz[0], matriz[1]), axis = 0)

def gerador_matriz(n_estagios, rao, d, cia, cio):
    
    matriz = []    
    for estagio_i in range(1, n_estagios + 1):        
        if estagio_i == 1:
            if type(d) == list:
                linha_i = np.concatenate(([rao + d[estagio_i - 1], -d[estagio_i]],
                                          np.zeros(n_estagios - estagio_i - 1), [cia * rao]), axis = 0)
            else:
                linha_i = np.concatenate(([rao + d, -d], np.zeros(n_estagios - estagio_i - 1), [cia * rao]), axis = 0)            
        elif estagio_i == n_estagios:
            if type(d) == list:
                linha_i = np.concatenate((np.zeros(n_estagios - 2), [-rao, rao + d[n_estagios - 1]], [cio]), axis = 0)                
            else:
                linha_i = np.concatenate((np.zeros(n_estagios - 2), [-rao, rao + d], [cio]), axis = 0)            
        else:
            if type(d) == list:
                linha_i = np.concatenate(
                          (np.zeros(estagio_i - 2), [-rao, rao + d[estagio_i - 1], -d[estagio_i]],
                           np.zeros(n_estagios - estagio_i)), axis = 0)
            else:
                linha_i = np.concatenate(
                          (np.zeros(estagio_i - 2), [-rao, rao + d, -d], np.zeros(n_estagios - estagio_i)), axis = 0)            
        matriz.append(linha_i)
        
    matriz = junta_matriz(matriz, n_estagios).reshape(n_estagios, n_estagios + 1)    
    return matriz

In [6]:
gerador_matriz(10, 2, 2, 3, 0)

array([[ 4., -2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6.],
       [-2.,  4., -2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -2.,  4., -2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -2.,  4., -2.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -2.,  4., -2.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -2.,  4., -2.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0., -2.,  4., -2.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0., -2.,  4., -2.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., -2.,  4., -2.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -2.,  4.,  0.]])

In [8]:
# Funções para escalonamento da matriz do balanço de massa

def muda_linha (li, lj, i): # 'lj' é a linha a ser modificada, 'li' a modificadora e 'i' é o elemento a ser zerado

    li_modificador = []
    
    for elemento in li:
        elemento = elemento * lj[i] / li[i]
        li_modificador.append(elemento)    

    for elemento in range(len(lj)):
        lj[elemento] = lj[elemento] - li_modificador[elemento]
        
    return lj

def escalona1 (matriz): #ela vai escalonar a matriz até zerar a diagonal de baixo

    n = 0
    while n < (len(matriz) - 1): #vai passar p/ próximo pivô
        l = 1
        while (l + n) < (len(matriz)): # vai zerar todos os termos de uma coluna que estão abaixo da diagonal
            matriz[n + l] = muda_linha (matriz[n], matriz[n + l], n)
            l += 1
        l = 1
        n += 1
        
    return matriz
    
def escalona2 (matriz): #ela vai escalonar a diagonal de cima, partindo da diagonal de baixo zerada
    
    matriz = escalona1(matriz)    
    n = len(matriz) - 1
    while n > 0:
        l = 1
        while l <= n: #vai passar p/ pivô anterior
            matriz[n - l] = muda_linha (matriz[n], matriz[n - l], n)
            l += 1
        l = 1
        n -= 1

    return matriz

def resolve_sistema (n_estagios, rao, d, cia, cio):
    
    matriz = escalona2(gerador_matriz(n_estagios, rao, d, cia, cio))
    concentraçoes_aquoso = []    
    
    for linha in range(len(matriz)):
        matriz[linha][-1] /= matriz[linha][linha]
        concentraçoes_aquoso.append(matriz[linha][-1])         
        
    return concentraçoes_aquoso

In [9]:
resolve_sistema(10, 2, 2, 5, 0)

[4.545454545454545,
 4.090909090909091,
 3.6363636363636367,
 3.1818181818181825,
 2.7272727272727284,
 2.272727272727274,
 1.8181818181818188,
 1.363636363636364,
 0.9090909090909093,
 0.4545454545454546]